# Open Street Map Project - Austin, TX

For this exercise, I worked with the Open Street Maps project data for city of Austin. The datafile was approximately 800 MB.
I started by loading the Python library for parsing XML files incrementally (cElementTree). I also included the Geocoder library as I used to calculate zipcode from latitude and longitude information available for nodes with certain specific features that interested me. In addition, I imported some more library to help with several other tasks as part of the data wrangling process including regular expressions, json (for file writing), pretty print (to print results that can be read easily), string (for string manipulation), ast (to convert a string variable into a dictionary during the file read process), and pymongo for communicating with mongodb. 

In [2]:
import xml.etree.cElementTree as ET
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
#from googlemaps import GoogleMaps
import time
import codecs
import json
import re
import string
import pprint
from pymongo import MongoClient
import pymongo
import ast
import os

## Data Cleaning and Wrangling

I defined the regular expressions and three list variables in the next section. I use these list variables to narrow down the list of features I want to focus for the node and way tags. In addition, my initial analysis suggested that while we are focusing on OSM data from Austin, many entries included the names for respective sub-divisions within Austin and they are all valid entries as many people refer by those names. As this information provided more detail on the locale for the respective entries, I chose to include te range of values within a defined list.

In [3]:
zipcode_re = re.compile(r'^[0-9]{5}')
city_comma_re = re.compile(r'^[A-Za-z ]+,')
count = 0
street_types = {}

list_of_cities = ['Austin','Cedar Park','Round Rock','West Lake Hills','Pflugerville','Sunset Valley','Del Valle']
node_tags_to_save = ["amenity","cuisine","public_transport","highway","shelter","shop","golf","emergency","name",\
               "bicycle_parking","man_made","sport","religion","website","building"]
way_tags_to_save = ["amenity","bicycle","bridge","golf","height","highway","name","landuse"]
zipcode_to_calc = ["amenity","cuisine","public_transport","highway","shelter","shop","golf","bicycle_parking",\
                       "sport","religion","building"]
node_tags = ['id']
creation_tags = ['changeset', 'uid', 'timestamp', 'version', 'user']

In the next step, just as in the exercises in class, I started with the fields which I expected to have an order - city, zipcode, county, etc. In my case I focused on two of the address fields - city and zipcode. The city field values contained two kinds of anomalies. The first was the city was suffixed by the abbreviation for the state (eg., Austin, TX) and the second group included address values that were suffixed by the city (eg., 614 W. 6th Street, Austin). The name of a city typically does not contain any number and therefore a regular expression that includes only letters in upper or lower case and spaces (city names can be two words or more) will help segregate the anomalies as they contain street numbers or commas. For the first group, the portion of the string before the first comma was used as the city value. For the second category, the portion of the string after the last comma was used as the city value.

In [4]:
def format_city(city_val):
    if city_val in list_of_cities or city_val.title() in list_of_cities:
        city = city_val.title()
    elif city_comma_re.search(city_val):
        city = city_val[:string.find(city_val,",")]
    elif city_val[string.find(city_val,",")+2:] in list_of_cities:
        city = tag.attrib["v"][string.find(city_val,",")+2:]
    if city_val.endswith(", TX") or city_val.endswith(", tx"):
        state = "TX"
    if state != None:
        return (city,state)
    else:
        return (city,)

The zipcode values also included two kinds of anomalies. The first group included entries where the five digit zipcode was suffixed with the four digit PO box separated by a dash(-). The second group included records that were prefixedwith "TX, ". I wrote a regular expression where the target value must start with five consecutive numbers. This would include proper five digit zipcode entries as well as the anomalies from the first group. So I obtained a substring comprising of the first five characters for this group which would return the five-digit zipcode, whether it is an anomaly or not. For the second category, I took a similar approach but this time I obtined a substring comprising of the fourth through the eigth character.

In [5]:
def format_zip(zip_val):
    if zipcode_re.search(zip_val):
        zip = zip_val[0:5]
    elif tag.attrib["v"].startswith("TX "):
        zip = zip_val[3:8]
        state = "TX"
    if state != None:
        return (zip,state)
    else:
        return (zip,)

As with the housenumber, the street categories though highly variable looked reasonable. Unfortunately, given the variety in the data, I considered this data to be extremely difficult to validate against any master data set or apply any kind of generalization rules. So I consumed this data as is.
Most of the header information are similar between the node and way tags except for the latitude and longitude, which is only available for the nodes. The location information was available for a significant number of the node entries and I tried to calculate the zip code for the ones that lacked address information. While I was partially successful, I faced challenges when calculating the zip code in bulk as the remote server caps the number of successive API calls within an interval of time and when violated issues "HTTP 429: too many requests" exception (http://stackoverflow.com/questions/22786068/how-to-avoid-http-error-429-too-many-requests-python).
The location information associated with the node tags made me more interested in the features for the nodes dataset which is evident from the defined lists in the previous section.

In [10]:
def reshape_tag(elem):
    
    node_ref = []
    node_info = {}
    creation_info = {}
    address_info = {}
    
    for record in elem.attrib:
        if record in node_tags:
            node_info[record] = elem.attrib[record]
        elif record in creation_tags:
            creation_info[record] = elem.attrib[record]
    for tag in elem.iter("tag"):
        if tag.attrib["k"].startswith("addr:city"):
            address_info["city"] = format_city(tag.attrib["v"])[0]
            if len(format_city(tag.attrib["v"])) > 1: address_info["state"] = format_city(tag.attrib["v"])[1]
        elif tag.attrib["k"].startswith("addr:postcode"):
            address_info["zipcode"] = format_zip(tag.attrib["v"])[0]
            if len(format_zip(tag.attrib["v"])) > 1: address_info["state"] = format_zip(tag.attrib["v"])[1]
        elif tag.attrib["k"].startswith("addr:state"):
            address_info["state"] = tag.attrib["v"]
        elif tag.attrib["k"].startswith("addr:street"):
            address_info["street"] = tag.attrib["v"]
        elif elem.tag == "node" and tag.attrib["k"] in node_tags_to_save:
            if tag.attrib["k"] == "cuisine": node_info['cuisine'] = tag.attrib["v"].split(";")
            else: node_info[tag.attrib["k"]] = tag.attrib["v"]
        elif elem.tag == "way" and tag.attrib["k"] in way_tags_to_save:
            node_info[tag.attrib["k"]] = tag.attrib["v"]
            
    for tag in elem.iter("nd"):
        node_ref.append(tag.attrib["ref"])
            
    node_info['created'] = creation_info.copy()
    if elem.tag == "node": node_info['location'] = [float(elem.attrib["lat"]),float(elem.attrib["lon"])]

    for tag in zipcode_to_calc:
        if tag in node_info:
            if "zipcode" not in address_info:
                try:
                    geolocator = Nominatim()
                    location = geolocator.reverse(elem.attrib["lat"]+","+elem.attrib["lon"], timeout=10000)
                    #time.sleep(0.5)
                    address_info["zipcode"] = location.raw["address"]["postcode"]
                except: pass
    if address_info != {}: node_info['address'] = address_info.copy()
        
    if elem.tag == "way": node_info["node_ref"] = node_ref[:]
        
    return node_info

In this section, I process the file downloaded using the Overpass API. I accumulate the node entries in the "OSM_data_nodes.json" and the way entries in "OSM_data_ways.json".

In [11]:
FILENAME = "map"

all_tags = {}

all_nodes = []
all_ways = []

file_out_nodes = "OSM_data_nodes.json"
file_out_ways = "OSM_data_ways.json"

with open(FILENAME,'r') as fl:
    for event, elem in ET.iterparse(fl):
        if elem.tag not in all_tags:
            all_tags[elem.tag] = 1
        else:
            all_tags[elem.tag] += 1
        if elem.tag == "node":
            all_nodes.append(reshape_tag(elem))
        elif elem.tag == "way":
            all_ways.append(reshape_tag(elem))
                    


with codecs.open(file_out_nodes, "w") as fon:
    for record in all_nodes:
        fon.write(json.dumps(record) + "\n")
            
with codecs.open(file_out_ways, "w") as fow:
    for record in all_ways:
        fow.write(json.dumps(record) + "\n")

## Gross Statistics about the Data

The file size for the "nodes" and "ways" dataset is given below.

In [120]:
print "Nodes file size: " + str(os.path.getsize("OSM_data_nodes.json")) + " Bytes"
print "Ways file size: " + str(os.path.getsize("OSM_data_ways.json")) + " Bytes"

Nodes file size: 630034550 Bytes
Ways file size: 120743393 Bytes


I establish the DB connection here.

In [6]:
client = MongoClient('localhost:27017')
db = client.OSM

I insert all the node records into the "nodes" collection.

In [42]:
for record in all_nodes:
    db.nodes.insert_one(record)

I insert all the way records into the "ways" collection.

In [19]:
for record in all_ways:
    db.ways.insert_one(record)

The nodes dataset contain the location details for each and every node, so I added a spatial index so that I can issue spatial queries.

In [54]:
db.nodes.ensure_index([('location',pymongo.GEO2D)])

/Users/ambarishbanerjee/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
  if __name__ == '__main__':


u'location_2d'

I started by querying the "nodes" collection to see how many nodes were in the dataset. The result shows our dataset contained approximately 3.15 million nodes.

In [110]:
pipeline_all_node_count = [{"$group":{"_id":"All Nodes","count":{"$sum":1}}},{"$project":{"_id":0,"count":1}}]
result_all_node_count = [doc for doc in db.nodes.aggregate(pipeline_all_node_count)]
pprint.pprint(result_all_node_count)

[{u'count': 3150468}]


Next, I did the same with the "ways" collection and found that there were approximately 340,000 ways in our dataset.

In [111]:
pipeline_all_way_count = [{"$group":{"_id":"All Ways","count":{"$sum":1}}},{"$project":{"_id":0,"count":1}}]
result_all_way_count = [doc for doc in db.ways.aggregate(pipeline_all_way_count)]
pprint.pprint(result_all_way_count)

[{u'count': 339908}]


Next I was curious to see the top three contributors for the "nodes" dataset and determine if the list was the same for the ways dataset. I noticed that the lists were the same with identical ordering in the top three ranks for both datasets.

In [59]:
pipeline_highest_contrib = [{"$group":{"_id":"$created.user","count":{"$sum":1}}},{"$sort":{"count":-1}}]
result_highest_contrib = [doc for doc in db.nodes.aggregate(pipeline_highest_contrib)]
pprint.pprint(result_highest_contrib[:3])
print ""

pipeline_highest_contrib_way = [{"$group":{"_id":"$created.user","count":{"$sum":1}}},{"$sort":{"count":-1}}]
result_highest_contrib_way = [doc for doc in db.ways.aggregate(pipeline_highest_contrib_way)]
pprint.pprint(result_highest_contrib_way[:3])

[{u'_id': u'patisilva_atxbuildings', u'count': 1472202},
 {u'_id': u'ccjjmartin_atxbuildings', u'count': 433859},
 {u'_id': u'ccjjmartin__atxbuildings', u'count': 246389}]

[{u'_id': u'patisilva_atxbuildings', u'count': 161307},
 {u'_id': u'ccjjmartin_atxbuildings', u'count': 39961},
 {u'_id': u'ccjjmartin__atxbuildings', u'count': 24059}]


Next, I wanted to see how many unique contributors we had for the "nodes" and "ways" datasets/collections. There were 704 unique contributors for the "nodes" collection and 529 for the "ways" collection.

In [60]:
pipeline_unique_users = [{"$group":{"_id":"Unique Users","unique_users":{"$addToSet":"$created.user"}}},\
                        {"$unwind":"$unique_users"},{"$group":{"_id":"Total Users","count":{"$sum":1}}},\
                         {"$project":{"_id":0,"count":1}}]
result_unique_users = [doc for doc in db.nodes.aggregate(pipeline_unique_users)]
pprint.pprint(result_unique_users)
print ""

pipeline_unique_users_ways = [{"$group":{"_id":"Unique Users","unique_users":{"$addToSet":"$created.user"}}},\
                        {"$unwind":"$unique_users"},{"$group":{"_id":"Total Users","count":{"$sum":1}}},\
                         {"$project":{"_id":0,"count":1}}]
result_unique_users_ways = [doc for doc in db.ways.aggregate(pipeline_unique_users_ways)]
pprint.pprint(result_unique_users_ways)

[{u'count': 704}]

[{u'count': 529}]


## A Few Interesting Highlights about the Austin OSM Data

In this query, I wanted to determine the top five zipcodes that had the highest concentration of restaurants. 78704 ranked highest on the list. 
PS > There were several instances for which the address fields were unknown and hence these were filtered from our recordset.

In [61]:
pipeline_zipcode_restaurant = [{"$match":{"cuisine":{"$exists":1}}},{"$match":{"address.zipcode":{"$exists":1}}},\
                               {"$group":{"_id":"$address.zipcode","count":{"$sum":1}}},{"$sort":{"count":-1}}]
result = [doc for doc in db.nodes.aggregate(pipeline_zipcode_restaurant)]
pprint.pprint(result[:5])

[{u'_id': u'78704', u'count': 16},
 {u'_id': u'78757', u'count': 11},
 {u'_id': u'78741', u'count': 9},
 {u'_id': u'78702', u'count': 8},
 {u'_id': u'78705', u'count': 7}]


In this query, I tried to obtain the five most popular cusine in Austin. Interestingly, "Mexican" ranked highest in this list.

In [66]:
pipeline_popular_cuisine = [{"$match":{"cuisine":{"$exists":1}}},\
                            {"$unwind":"$cuisine"},{"$group":{"_id":"$cuisine","count":{"$sum":1}}},{"$sort":{"count":-1}}]
result_pop_cuisine = [doc for doc in db.nodes.aggregate(pipeline_popular_cuisine)]
pprint.pprint(result_pop_cuisine[:5])

[{u'_id': u'mexican', u'count': 62},
 {u'_id': u'sandwich', u'count': 34},
 {u'_id': u'burger', u'count': 32},
 {u'_id': u'pizza', u'count': 28},
 {u'_id': u'coffee_shop', u'count': 19}]


Lastly, I wanted to look at some descriptive statistics for the "ways" that had references to nodes. While the highest number of nodes referenced in a single "way" tag was ~1800, the average was about 10.

In [113]:
max_node_way_pipeline = [{"$unwind":"$node_ref"},{"$group":{"_id":"$id","count":{"$sum":1}}},{"$sort":{"count":-1}},\
                         {"$limit":1}]
result_max_node_way = [doc for doc in db.ways.aggregate(max_node_way_pipeline)]
pprint.pprint(result_max_node_way)
print ""

avg_node_way_pipeline = [{"$unwind":"$node_ref"},{"$group":{"_id":"$id","count":{"$sum":1}}},\
                         {"$group":{"_id":"Average Node Count","avgCount":{"$avg":"$count"}}}]
result_avg_node_way = [doc for doc in db.ways.aggregate(avg_node_way_pipeline)]
pprint.pprint(result_avg_node_way)

[{u'_id': u'33171423', u'count': 1806}]

[{u'_id': u'Average Node Count', u'avgCount': 10.230924250091201}]


## Geospatial Querying

I wanted to locate the five closest restaurants by the University of Texas. For this, I used the spatial index that was created earlier on the "location" field. Interestingly, the top restaurant's cusine includes "\*" implying everything. 
Side note: I know from experience that thisis true as this establishment is part of one of the dormitories that has a food court which serves just about everything.

In [108]:
university_location = {"amenity":{"$exists":1},"amenity":"university"}
university_loc_proj = {"_id":0,"location":1}
result_univ_loc = [doc for doc in db.nodes.find(university_location,university_loc_proj)]
univ_loc = result_univ_loc[:1][0]["location"]


query_location_based = {"location":{"$near":univ_loc},"cuisine":{"$exists":1}}
project_location_based = {"_id":0,"name":1,"cuisine":2}
result_location_based = [doc for doc in db.nodes.find(query_location_based,project_location_based)]
pprint.pprint(result_location_based[:5])

[{u'cuisine': [u'*'], u'name': u'Dobie Tower Food Court'},
 {u'cuisine': [u'asian'], u'name': u"Emo's Kitchen"},
 {u'cuisine': [u'sandwich'], u'name': u'Bite Mi'},
 {u'cuisine': [u'thai'], u'name': u'Thai How Are You'},
 {u'cuisine': [u'indian'], u'name': u"Teji's"}]


## Suggested Improvements and Conclusion

One thing I felt could have added a lot of value to the OSM data is the availability of zipcode or some measure of locale that is easily relatable. The information provided as part of the OSM data was most likely obtained using mobile devices. Most devices that allow geotagging can determine the zipcode and rough approximations for the street address. Users who are providing the information can greatly help the consumers of this information by providing some geographic measure of the locale along with GPS coordinates. This would facilitate the grouping of information by location in a more logical sense. Possible benefits of this enhancement can include, zip code with the highest number of schools or parks or any other amenity of choice to the user. 
I attempted to calclulate the same but restrictions enforced on the server hosting the API prevented me from doing so. So I thought if this information was available, it would have added considerable value to the data.